<a href="https://colab.research.google.com/github/naro-naro/Colab/blob/main/230416_practicalNLP_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##### 데이터 다운로드
!pip install datasets   # Package install

from datasets import load_dataset   # Huggingface 데이터셋 패키지 import
data = load_dataset("sepidmnorozy/Korean_sentiment")    # 데이터 다운로드

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 3.7 MB/s eta 0:00:00


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/sepidmnorozy___csv/sepidmnorozy--Korean_sentiment-c4fbcaaf8f2ffcfa/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
##### 데이터 구조 훑어보기
# print("Data type: ", type(data))    # 데이터 타입 확인
# print("Data structure: ", data)     # 데이터 구조 확인
# print("Data keys: ", data.keys())   # 데이터 키 확인

print(data['train'][0])   # 실제 데이터 확인

{'label': 1, 'text': '역시 명작 어렸을때 봤을때도 재밌었고 지금 봐도 몇억배 이상으로 재밌어요'}


In [ ]:
##### 테스트 세트 만들기
train_data = data['train']
dev_data = data['validation']
test_data = data['test']

#print(train_data)
#print(dev_data)
#print(test_data)

In [ ]:
##### 텍스트 데이터 특성
# import matplotlib.pyplot as plt
# plt.hist(data['train']['label'], color='red')
# plt.show()
# plt.hist(data['validation']['label'], color='blue')
# plt.show()
# plt.hist(data['test']['label'], color='green')
# plt.show()

In [ ]:
##### 텍스트와 범주형 특성 다루기 (Bag-of-words 방식으로 벡터화 하는 법 구현)
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer       # CountVectorizer: Bag of words 벡터화 구현을 하기 위한 클래스

vectorizer = TfidfVectorizer(strip_accents='unicode', token_pattern=r"(?u)\b\w\w+\b|'\w+", ngram_range=(1,2))      # 데이터를 벡터화 해주는 모델 #토큰 패턴을 부여하여 '큰'과 '만'도 보존되도록
vectorizer.fit(train_data['text'])  # 텍스트 문서 모음을 토큰 수의 행렬로 변환

print(vectorizer.vocabulary_)       # 텍스트 문서에 나타난 어휘의 집합을 출력
print(len(vectorizer.vocabulary_))  # 텍스트 문서에 나타난 어휘 집합의 길이를 출력

train_vectors = vectorizer.transform(train_data['text'])    # 학습 데이터를 숫자로 변환
dev_vectors = vectorizer.transform(dev_data['text'])        # 검증 데이터를 숫자로 변환
test_vectors = vectorizer.transform(test_data['text'])      # 테스트 데이터를 숫자로 변환

# print(train_vectors)      # transform 된 결과 확인


##### 텍스트와 범주형 특성 다루기 (Bag-of-words 방식으로 벡터화 한 것 확인하는 방법 구현)
sample_num = 0                            # 확인하고 싶은 샘플 번호
sample_origin = train_data[sample_num]    # 확인하고 싶은 샘플의 원래 문장
sample_transform = train_vectors[sample_num]    # 확인하고 싶은 샘플의 transform된 결과
sample_inverse_transform = vectorizer.inverse_transform(sample_transform) # 확인하고 싶은 샘플의 transform된 결과를 다시 단어의 조합으로 바꾼 결과
print("Original Input:{}\nTransformed: {}\nInv-transformed: {}".format(sample_origin, sample_transform, sample_inverse_transform))    # 출력 --'큰', '만'등이 없어짐 : 영어를 기준으로 한 언어라서

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Original Input:{'label': 1, 'text': '역시 명작 어렸을때 봤을때도 재밌었고 지금 봐도 몇억배 이상으로 재밌어요'}
Transformed:   (0, 270502)	0.21328371562377132
  (0, 270438)	0.15308102241524757
  (0, 248903)	0.26776994083482364
  (0, 248891)	0.2213534588724557
  (0, 248760)	0.16406450795999622
  (0, 228953)	0.26776994083482364
  (0, 228946)	0.2254960242584748
  (0, 194932)	0.26776994083482364
  (0, 194819)	0.1369479147602931
  (0, 185195)	0.26776994083482364
  (0, 185181)	0.19411034626692955
  (0, 132826)	0.26776994083482364
  (0, 132819)	0.2333929601893755
  (0, 130883)	0.26776994083482364
  (0, 130823)	0.15143507006598256
  (0, 103420)	0.26776994083482364
  (0, 103419)	0.26776994083482364
  (0, 102675)	0.26776994083482364
  (0, 102634)	0.15864488839421678
Inv-transformed: [array(['지금 봐도', '지금', '재밌었고 지금', '재밌었고',
       '재밌어요', '이상으로 재밌어요', '이상으로', '역시 명작',
       '역시', '어렸을때 봤을때도', '어렸을때',
       '봤을때도 재밌었고', '봤을때도', '봐도 몇ᄋ

In [ ]:
##### 훈련세트에서 훈련하고 평가하기
from sklearn.svm import LinearSVC     #Linear kernel SVM을 사용하기 위한 클래스

svm = LinearSVC()     # 모델 정의
svm.fit(train_vectors, train_data['label'])     # 모델 학습

print(svm.coef_)      # weights (w)
print(svm.intercept_)     # bias (b)

[[-0.19682362 -0.17832739 -0.17832739 ...  0.09242357 -0.02698433
  -0.02698433]]
[-0.06938533]


In [ ]:
##### 교차 검증을 사용한 평가
# from sklearn.model_selection import cross_val_score

# all_data = train_data['text']+dev_data['text']+test_data['text']      # all_data = train_data + dev_data + test_data
# all_label = train_data['label']+dev_data['label']+test_data['label']  # all_label은 train_data, dev_data, test_date의 레이블을 모두 합한 것
# all_vectors = vectorizer.transform(all_data)    # all_data를 숫자로 변환
# scores = cross_val_score(svm, all_vectors, all_label, cv=5)  # 5-cross validation

# print("All scores:", scores)  # 전체 점수 출력
# print("Average: {:.2f}%".format(scores.mean()*100)) # 다섯 번의 스코어 평균 출력
# print("Standard deviation: {:.6f}".format(scores.std()))  # 표준편차 출력

In [ ]:
##### 그리드 탐색
# from sklearn.model_selection import GridSearchCV

# param_grid = [{'max_iter':[500, 1000, 5000], 'C': [1, 10, 100]}]        # 학습 max_iter와 C를 변동 하이퍼파라미터로 설정
# grid_search = GridSearchCV(svm, param_grid, cv=3)     # 하이퍼파라미터 서치 할 모델 설정, 각 모델별로 5-cross validation 실행
# grid_search.fit(train_vectors, train_data['label'])   # transformed train data로 학습

# print(grid_search.cv_results_['mean_test_score'])     # 파라미터 서치 결과
# print(grid_search.best_params_)   #최고의 모델 파라미터

# #파라미터 서치 결과 출력
# for mean_score, params in zip(grid_search.cv_results_['mean_test_score'], grid_search.cv_results_['params']):
#   print(mean_score, params)

In [ ]:
##### 테스트 세트로 시스템 평가하기
from sklearn.metrics import accuracy_score    # Accuracy 측정 함수 import

final_model = svm #grid_search.best_estimator_     # grid search에 의해 정해진 제일 좋은 하이퍼파라미터

pred_results = final_model.predict(test_vectors)    # 최종 모델로 test 데이터 예측
accuracy = accuracy_score(test_data['label'], pred_results)   # 정확도 측정

print("Accuracy: {:.2f}%".format(accuracy*100))   # 정확도 출력

Accuracy: 82.26%


In [ ]:
# connect google drive
from google.colab import drive
drive.mount('/content/drive')

# Download konlpy
%cd ./drive/MyDrive/Colab\ Notebooks/
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd ./Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab_light_220429.sh


Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks
fatal: destination path 'Mecab-ko-for-Google-Colab' already exists and is not an empty directory.
/content/drive/MyDrive/Colab Notebooks/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 41.4 MB/s eta 0:00:00
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2023-04-16 04:31:57--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::3403:4be7, 2406:da00:ff00::22c0:3470, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: ht

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

sentence = "실용자연어처리 실습 진행중 입니다."
print(mecab.morphs(sentence))
print(mecab.nouns(sentence))
print(mecab.pos(sentence))

['실용', '자연어', '처리', '실습', '진행', '중', '입니다', '.']
['실용', '자연어', '처리', '실습', '진행', '중']
[('실용', 'NNG'), ('자연어', 'NNG'), ('처리', 'NNG'), ('실습', 'NNG'), ('진행', 'NNG'), ('중', 'NNB'), ('입니다', 'VCP+EF'), ('.', 'SF')]


In [ ]:
train_data = data['train']
dev_data = data['validation']
test_data = data['test']

print(train_data)
print(dev_data)
print(test_data)

train_data = train_data.map(lambda example: {'label': example['label'], 'text': ' '.join(mecab.morphs(example['text']))})
dev_data = dev_data.map(lambda example: {'label': example['label'], 'text': " ".join(mecab.morphs(example['text']))})
test_data = test_data.map(lambda example: {'label': example['label'], 'text': " ".join(mecab.morphs(example['text']))})

print(train_data[0])

Dataset({
    features: ['label', 'text'],
    num_rows: 36000
})
Dataset({
    features: ['label', 'text'],
    num_rows: 1333
})
Dataset({
    features: ['label', 'text'],
    num_rows: 2667
})


Map:   0%|          | 0/36000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1333 [00:00<?, ? examples/s]

Map:   0%|          | 0/2667 [00:00<?, ? examples/s]

{'label': 1, 'text': '역시 명작 어렸 을 때 봤 을 때 도 재밌 었 고 지금 봐도 몇 억 배 이상 으로 재밌 어요'}


In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer     # CountVectorizer: Bag of words 벡터화 구현을 하기 위한 클래스

# vectorizer = CountVectorizer(strip_accents='unicode', token_pattern=r"(?u)\b\w\w+\b|'\w+")      # 데이터를 벡터화 해주는 모델
# vectorizer.fit(train_data['text'])  # 텍스트 문서 모음을 토큰 수의 행렬로 변환

# print(vectorizer.vocabulary_)       # 텍스트 문서에 나타난 어휘의 집합을 출력
# print(len(vectorizer.vocabulary_))  # 텍스트 문서에 나타난 어휘 집합의 길이를 출력

# train_vectors = vectorizer.transform(train_data['text'])    # 학습 데이터를 숫자로 변환
# dev_vectors = vectorizer.transform(dev_data['text'])        # 검증 데이터를 숫자로 변환
# test_vectors = vectorizer.transform(test_data['text'])      # 테스트 데이터를 숫자로 변환

# print(train_vectors)

불용어 처리와 카운트벡토라이저 동시 사용불가하므로 주석처리

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer     # CountVectorizer: Bag of words 벡터화 구현을 하기 위한 클래스

from konlpy.tag import Komoran

ko_stop_words = ['을','도', '고', '요']
vectorizer = CountVectorizer(stop_words = ko_stop_words)      # 데이터를 벡터화 해주는 모델
vectorizer.fit(train_data['text'])  # 텍스트 문서 모음을 토큰 수의 행렬로 변환

print(vectorizer.vocabulary_)       # 텍스트 문서에 나타난 어휘의 집합을 출력
print(len(vectorizer.vocabulary_))  # 텍스트 문서에 나타난 어휘 집합의 길이를 출력

train_vectors = vectorizer.transform(train_data['text'])    # 학습 데이터를 숫자로 변환
dev_vectors = vectorizer.transform(dev_data['text'])        # 검증 데이터를 숫자로 변환
test_vectors = vectorizer.transform(test_data['text'])      # 테스트 데이터를 숫자로 변환

print(train_vectors)

{'역시': 16019, '명작': 9038, '어렸': 15419, '재밌': 19199, '지금': 20586, '봐도': 11076, '이상': 18128, '으로': 17661, '어요': 15490, '이런': 18009, '영화': 16264, '연습': 16085, '라도': 7638, '만들': 8498, '양심': 15308, '세대': 12883, '우주': 17068, '넘나드': 5184, '스카이워커': 13565, '비극': 11591, '진짜': 20898, '20': 114, '제작': 19869, '다는': 5787, '믿기': 9823, '는다': 5629, '천재': 21483, '감독': 1900, '소소': 13052, '재미': 19194, '모아서': 9120, '만든': 8492, '느낌': 5586, '랄까': 7721, 'ㅎㅎ': 1190, '방황': 10380, '청춘': 21576, '너무': 5163, '느끼': 5581, '트라볼타': 22978, '음악': 17805, '별로': 10780, '스토리': 13643, '막장': 8455, '이건': 17886, '쓰레기': 14385, 'ㅋㅋㅋ': 1178, 'ㅋㅋ': 1177, '원작': 17238, '새로운': 12394, '재해석': 19242, '특수': 23038, '효과': 25024, '전반': 19434, '정말': 19637, '웃긴': 17163, '네요': 5235, '추천': 21846, '근데': 3720, '99': 264, '나온': 4732, '이제': 18293, '나왔': 4761, '는데': 5638, '수많': 13274, '최악': 21757, '하나': 23834, '주제': 20335, '감동': 1902, '심사': 14192, '대전': 6265, '당시': 6130, '97': 262, '극장': 3700, '에서': 15779, '감명': 1905, '2010': 127, '서울': 12613, '도시': 6

In [ ]:
sample_num = 0                            # 확인하고 싶은 샘플 번호
sample_origin = train_data[sample_num]    # 확인하고 싶은 샘플의 원래 문장
sample_transform = train_vectors[sample_num]    # 확인하고 싶은 샘플의 transform된 결과
sample_inverse_transform = vectorizer.inverse_transform(sample_transform) # 확인하고 싶은 샘플의 transform된 결과를 다시 단어의 조합으로 바꾼 결과
print("Original Input:{}\nTransformed: {}\nInv-transformed: {}".format(sample_origin, sample_transform, sample_inverse_transform))

Original Input:{'label': 1, 'text': '역시 명작 어렸 을 때 봤 을 때 도 재밌 었 고 지금 봐도 몇 억 배 이상 으로 재밌 어요'}
Transformed:   (0, 9038)	1
  (0, 11076)	1
  (0, 15419)	1
  (0, 15490)	1
  (0, 16019)	1
  (0, 17661)	1
  (0, 18128)	1
  (0, 19199)	2
  (0, 20586)	1
Inv-transformed: [array(['명작', '봐도', '어렸', '어요', '역시', '으로', '이상', '재밌', '지금'], dtype='<U25')]


In [ ]:
##### 훈련세트에서 훈련하고 평가하기
from sklearn.svm import LinearSVC     #Linear kernel SVM을 사용하기 위한 클래스

svm = LinearSVC()     # 모델 정의
svm.fit(train_vectors, train_data['label'])     # 모델 학습

print(svm.coef_)      # weights (w)
print(svm.intercept_)     # bias (b)

[[-0.10708698 -0.14791585  0.         ... -0.06338703  0.
  -0.60751792]]
[-0.0887189]


/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [ ]:
##### 교차 검증을 사용한 평가
# from sklearn.model_selection import cross_val_score

# all_data = train_data['text']+dev_data['text']+test_data['text']      # all_data = train_data + dev_data + test_data
# all_label = train_data['label']+dev_data['label']+test_data['label']  # all_label은 train_data, dev_data, test_date의 레이블을 모두 합한 것
# all_vectors = vectorizer.transform(all_data)    # all_data를 숫자로 변환
# scores = cross_val_score(svm, all_vectors, all_label, cv=5)  # 5-cross validation

# print("All scores:", scores)  # 전체 점수 출력
# print("Average: {:.2f}%".format(scores.mean()*100)) # 다섯 번의 스코어 평균 출력
# print("Standard deviation: {:.6f}".format(scores.std()))  # 표준편차 출력

In [ ]:
##### 그리드 탐색
# from sklearn.model_selection import GridSearchCV

# param_grid = [{'max_iter':[500, 1000, 5000], 'C': [1, 10, 100]}]        # 학습 max_iter와 C를 변동 하이퍼파라미터로 설정
# grid_search = GridSearchCV(svm, param_grid, cv=3)     # 하이퍼파라미터 서치 할 모델 설정, 각 모델별로 5-cross validation 실행
# grid_search.fit(train_vectors, train_data['label'])   # transformed train data로 학습

# print(grid_search.cv_results_['mean_test_score'])     # 파라미터 서치 결과
# print(grid_search.best_params_)   #최고의 모델 파라미터

# #파라미터 서치 결과 출력
# for mean_score, params in zip(grid_search.cv_results_['mean_test_score'], grid_search.cv_results_['params']):
#   print(mean_score, params)

In [ ]:
##### 테스트 세트로 시스템 평가하기
from sklearn.metrics import accuracy_score    # Accuracy 측정 함수 import

final_model = svm #grid_search.best_estimator_     # grid search에 의해 정해진 제일 좋은 하이퍼파라미터

pred_results = final_model.predict(test_vectors)    # 최종 모델로 test 데이터 예측
accuracy = accuracy_score(test_data['label'], pred_results)   # 정확도 측정

print("Accuracy: {:.2f}%".format(accuracy*100))   # 정확도 출력

Accuracy: 79.57%
